In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

#### Verificando dados de usuário

In [2]:
#### Verificando dados de usuário
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4543,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1561,"completed":34475,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

In [113]:
# Selecionar data da pesquisa
# date_from = "2023-11-1"
# date_to = "2023-11-3"

# date_from = "2023-11-3"
# date_to = "2023-11-5"

# date_from = "2023-11-5"
# date_to = "2023-11-7"

# date_from = "2023-11-7"
# date_to = "2023-11-9"

# date_from = "2023-11-9"
# date_to = "2023-11-11"

# date_from = "2023-11-11"
# date_to = "2023-11-13"

# date_from = "2023-11-13"
# date_to = "2023-11-15"

# date_from = "2023-11-15"
# date_to = "2023-11-17"

# date_from = "2023-11-17"
# date_to = "2023-11-19"

# date_from = "2023-11-19"
# date_to = "2023-11-21"

# date_from = "2023-11-21"
# date_to = "2023-11-23"

# date_from = "2023-11-23"
# date_to = "2023-11-25"

# date_from = "2023-11-25"
# date_to = "2023-11-27"

# date_from = "2023-11-27"
# date_to = "2023-11-29"

# date_from = "2023-11-29"
# date_to = "2023-12-1"

# date_from = "2023-12-1"
# date_to = "2023-12-3"

# date_from = "2023-12-3"
# date_to = "2023-12-5"

date_from = "2023-12-5"
date_to = "2023-12-7"

# URL base da API
base_url = "https://api.mercadolibre.com/orders/search"

# Parâmetros iniciais
params = {
    "seller": "233632476",
    "order.date_closed.from": f"{date_from}T00:00:00.000-03:00",
    "order.date_closed.to": f"{date_to}T00:00:00.000-03:00",
    "limit": 50,
    "offset": 0,
}

headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

json_list = []

counter = 0

try:
    while True:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Lança uma exceção se a resposta não for bem-sucedida
        data = response.json()

        if "results" in data:
            json_list.extend(data["results"])
        else:
            break

        # Verifique se há mais páginas
        if "paging" in data:
            total_paging = data["paging"].get("total")
            if total_paging is None:
                break

            total_pages = math.ceil(total_paging / params["limit"])
            print(f"Total esperado de páginas: {counter + 1}/{total_pages}")
            print(f"Total de dados esperados: {total_paging}")
            print(f'Offset atual: {params["offset"]}')

            counter += 1
            # if params["offset"] >= total_paging:
            if params["offset"] > total_paging:
                break

            params["offset"] += params["limit"]
        else:
            break

except requests.exceptions.RequestException as req_err:
    print(f"Erro ao fazer a requisição para {base_url}: {req_err}")

except Exception as e:
    print(f"Erro não esperado: {e}")

print(f"Total de dados coletados: {len(json_list)}")

Total esperado de páginas: 1/5
Total de dados esperados: 223
Offset atual: 0
Total esperado de páginas: 2/5
Total de dados esperados: 223
Offset atual: 50
Total esperado de páginas: 3/5
Total de dados esperados: 223
Offset atual: 100
Total esperado de páginas: 4/5
Total de dados esperados: 223
Offset atual: 150
Total esperado de páginas: 5/5
Total de dados esperados: 223
Offset atual: 200
Total esperado de páginas: 6/5
Total de dados esperados: 223
Offset atual: 250
Total de dados coletados: 223


In [114]:
print(len(json_list))
json_list

223


[{'payments': [{'reason': 'Ukulele Concert Acústico Tagima Natural Mahogany Satin 23k',
    'status_code': None,
    'total_paid_amount': 224.2,
    'operation_type': 'regular_payment',
    'transaction_amount': 224.2,
    'transaction_amount_refunded': 0,
    'date_approved': '2023-12-04T23:12:37.000-04:00',
    'collector': {'id': 233632476},
    'coupon_id': None,
    'installments': 4,
    'authorization_code': '962108',
    'taxes_amount': 0,
    'id': 67975866319,
    'date_last_modified': '2023-12-04T23:12:43.000-04:00',
    'coupon_amount': 0,
    'available_actions': ['refund'],
    'shipping_cost': 0,
    'installment_amount': 56.05,
    'date_created': '2023-12-04T23:12:35.000-04:00',
    'activation_uri': None,
    'overpaid_amount': 0,
    'card_id': 9303422791,
    'status_detail': 'accredited',
    'issuer_id': '25',
    'payment_method_id': 'visa',
    'payment_type': 'credit_card',
    'deferred_period': None,
    'atm_transfer_reference': {'transaction_id': '4566240',

In [115]:
with open("orders.json", "w") as arquivo:
    json.dump(json_list, arquivo)

In [116]:
with open("orders.json", "r") as file:
    json_list = json.load(file)

json_list

[{'payments': [{'reason': 'Ukulele Concert Acústico Tagima Natural Mahogany Satin 23k',
    'status_code': None,
    'total_paid_amount': 224.2,
    'operation_type': 'regular_payment',
    'transaction_amount': 224.2,
    'transaction_amount_refunded': 0,
    'date_approved': '2023-12-04T23:12:37.000-04:00',
    'collector': {'id': 233632476},
    'coupon_id': None,
    'installments': 4,
    'authorization_code': '962108',
    'taxes_amount': 0,
    'id': 67975866319,
    'date_last_modified': '2023-12-04T23:12:43.000-04:00',
    'coupon_amount': 0,
    'available_actions': ['refund'],
    'shipping_cost': 0,
    'installment_amount': 56.05,
    'date_created': '2023-12-04T23:12:35.000-04:00',
    'activation_uri': None,
    'overpaid_amount': 0,
    'card_id': 9303422791,
    'status_detail': 'accredited',
    'issuer_id': '25',
    'payment_method_id': 'visa',
    'payment_type': 'credit_card',
    'deferred_period': None,
    'atm_transfer_reference': {'transaction_id': '4566240',

### Coletando dados de pagamento


In [117]:
resultados = []

for item in json_list:
    # Extrair os valores desejados
    payments = item.get("payments", [])
    status = item["status"]
    date_closed = item["date_closed"]
    pack_id = item["pack_id"]
    shipping = item["shipping"]
    order_items = item.get("order_items", [])
    fulfilled = item["fulfilled"]

    # Inicializa uma lista para armazenar os reasons de payments
    payment_reasons = []

    for payment in payments:
        reason = payment.get("status")
        date_approved = payment.get("date_approved")
        payment_reasons.append(reason)

    # Inicializa variável para armazenar shipping_id
    shipping_id = None

    # Verifica se há pelo menos um item em "shipping"
    if shipping:
        shipping_id = shipping.get("id")

    # Inicializa listas para armazenar informações específicas de order_items
    order_items_ids = []
    order_items_title = []
    order_items_variation_id = []
    order_items_variation_attributes = []
    order_items_seller_sku = []
    order_items_category_id = []
    quantity_list = []
    name_list = []
    id_list = []
    value_id_list = []
    value_name_list = []

    # Itera sobre os dicionários em order_items
    for order_item in order_items:
        item_info = order_item.get("item", {})
        item_id = item_info.get("id")
        title = item_info.get("title")
        variation_id = item_info.get("variation_id")
        seller_sku = item_info.get("seller_sku")
        quantity = order_item.get("quantity")
        category_id = item_info.get("category_id")
        variation_attributes = item_info.get("variation_attributes", [])

        # Adiciona as informações de variation_attributes a listas separadas
        if variation_attributes:
            for attribute in variation_attributes:
                name = attribute.get("name")
                id = attribute.get("id")
                value_id = attribute.get("value_id")
                value_name = attribute.get("value_name")

                name_list.append(name)
                id_list.append(id)
                value_id_list.append(value_id)
                value_name_list.append(value_name)
        else:
            # Se variation_attributes for vazio, adiciona valores nulos às listas
            name_list.append(None)
            id_list.append(None)
            value_id_list.append(None)
            value_name_list.append(None)

        # Adiciona as informações restantes às listas
        order_items_ids.append(item_id)
        order_items_title.append(title)
        order_items_variation_id.append(variation_id)
        order_items_seller_sku.append(seller_sku)
        quantity_list.append(quantity)
        order_items_category_id.append(category_id)

    # Adicionar os resultados à lista
    resultados.append(
        {
            # "payments": payments,
            "ml_code": order_items_ids[0] if order_items_ids else None,
            "payment_status": payment_reasons[0] if payment_reasons else None,
            "order_status": status,
            "shipping_id": shipping_id,
            "pack_id": pack_id,
            "title": order_items_title[0] if order_items_title else None,
            "variation_id": order_items_variation_id[0]
            if order_items_variation_id
            else None,
            "category_id": order_items_category_id[0]
            if order_items_category_id
            else None,
            "seller_sku": order_items_seller_sku[0] if order_items_seller_sku else None,
            "quantity": quantity_list[0] if quantity_list else None,
            "variation_name": name_list[0] if name_list else None,
            "variation_attributes_id": id_list[0] if id_list else None,
            "variation_value_id": value_id_list[0] if value_id_list else None,
            "variation_value_name": value_name_list[0] if value_name_list else None,
            "fulfilled": fulfilled,
            "date_approved": date_approved,
            "date_closed": date_closed,
        }
    )

# Exibir os resultados
pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(resultados)
df["variation_id"] = df["variation_id"].astype(str).apply(lambda x: x.rstrip(".0"))
df["pack_id"] = df["pack_id"].astype(str).apply(lambda x: x.rstrip(".0"))

print(df.shape)
df.head(10)

(223, 17)


,ml_code,payment_status,order_status,shipping_id,pack_id,title,variation_id,category_id,seller_sku,quantity,variation_name,variation_attributes_id,variation_value_id,variation_value_name,fulfilled,date_approved,date_closed
0,MLB2171831134,approved,paid,42870293191,nan,Ukulele Concert Acústico Tagima Natural Mahogany Satin 23k,174246496744,MLB202371,FULLC423KNS,1,Cor,COLOR,52005,Marrom,None,2023-12-04T23:12:37.000-04:00,2023-12-04T23:12:37.000-04:00
1,MLB1992567302,approved,paid,42870505194,nan,1 Par Baquetas De Bateria Liverpool Nirvana 5a Ou 5b Ou 7a,173585284762,MLB429603,FULLNI5BM,1,Tamanho,None,None,1 Par 5B,None,2023-12-04T23:59:53.000-04:00,2023-12-05T00:00:54.000-04:00
2,MLB3497394073,approved,paid,42870520258,nan,Hercules Pedestal De Microfone Ms120b Mini Girafa Cor Preto,nan,MLB49488,FULLHERCULESMS120B,1,None,None,None,None,None,2023-12-05T00:07:02.000-04:00,2023-12-05T00:07:02.000-04:00
3,MLB3474452659,approved,paid,42870434001,2000005128048233,Cabo P10 X P10 Santo Angelo Ninja 20ft - 6.10m,nan,MLB72745,P10NINJA20FT1,2,None,None,None,None,True,2023-12-05T00:26:37.000-04:00,2023-12-05T00:26:37.000-04:00
4,MLB3461876520,approved,paid,42870554311,2000005128274699,Encordoamento Sg Cordas 009 Para Guitarra - Kit Com 3 Jogos,nan,MLB278076,FULLSG5145TP,1,None,None,None,None,None,2023-12-05T02:19:10.000-04:00,2023-12-05T02:19:10.000-04:00
5,MLB4089283094,approved,paid,42850945042,nan,Faca Carne Refile Aereo Lam. Reta Larga Inox 35cm Bky203-14 Cor Amarelo,nan,MLB194032,C4BKY20314,1,None,None,None,None,None,2023-12-05T03:06:49.000-04:00,2023-12-05T03:07:50.000-04:00
6,MLB2075516288,approved,paid,42854968029,nan,Kit 6 Pares De Baquetas D Bateria Liverpool Nirvana 5a 5b 7a,173887022577,MLB429603,FULLLIVERPOOL6NI5AM,1,Tamanho,None,None,6 PARES 5A,None,2023-12-05T03:08:59.000-04:00,2023-12-05T03:10:00.000-04:00
7,MLB3427769549,approved,paid,42870706240,2000005118127889,Pandeiro Profissional Izzo Pele Preta 10 Corpo Abs 6480 Cor Preto,nan,MLB29319,FULLIZZO6480,1,None,None,None,None,None,2023-12-05T03:15:19.000-04:00,2023-12-05T03:15:20.000-04:00
8,MLB3472227063,approved,paid,42870719512,2000005128335699,Pandeiro 10 Polegadas Izzo 3440vd Verde Com Pele Preta Abs,nan,MLB29319,C43440VD,1,None,None,None,None,None,2023-12-05T03:27:48.000-04:00,2023-12-05T03:27:48.000-04:00
9,MLB3461876520,approved,paid,42859107215,nan,Encordoamento Sg Cordas 009 Para Guitarra - Kit Com 3 Jogos,nan,MLB278076,FULLSG5145TP,1,None,None,None,None,None,2023-12-05T03:32:05.000-04:00,2023-12-05T03:33:05.000-04:00


In [118]:
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in df.iterrows():
    # for index, row in df_resultado.iterrows():
    # insert_query = sql.SQL(
    #     "INSERT INTO ml_orders_hist (ml_code,category_id,variation_id,seller_sku,pack_id,quantity,title,order_status,payment_status,shipping_id,date_approved,date_closed,fulfilled) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s)"
    # )
    insert_query = sql.SQL(
        "INSERT INTO ml_orders (ml_code,category_id,variation_id,seller_sku,pack_id,quantity,title,order_status,payment_status,variation_name,variation_attributes_id,variation_value_id,variation_value_name,shipping_id,date_approved,date_closed,fulfilled) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)"
    )
    cursor.execute(
        insert_query,
        (
            row["ml_code"],
            row["category_id"],
            row["variation_id"],
            row["seller_sku"],
            row["pack_id"],
            row["quantity"],
            row["title"],
            row["order_status"],
            row["payment_status"],
            row["variation_name"],
            row["variation_attributes_id"],
            row["variation_value_id"],
            row["variation_value_name"],
            row["shipping_id"],
            row["date_approved"],
            row["date_closed"],
            row["fulfilled"],
        ),
    )

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


In [ ]:
# meta_cols = ["date_closed", "pack_id", "shipping", "order_items", "fulfilled"]
# df_payments = json_normalize(json_list, record_path=["payments"], meta=meta_cols)

# df_payments.columns

In [ ]:
# cols = ["date_approved", "status", "shipping", "fulfilled"]
# df_payments = df_payments[cols]

# col = {"status": "payment_status"}
# df_payments.rename(columns=col, inplace=True)

In [ ]:
# df_payments.head(1)

In [ ]:
# # Removendo valores nulos em date_approved
# print(df_payments.shape)
# df_payments = df_payments.dropna(subset=["date_approved"])
# print(df_payments.shape)

In [ ]:
# # Extraindo shipping_id
# df_payments["shipping_id"] = df_payments["shipping"].apply(lambda x: x["id"])
# df_payments["shipping_id"] = (
#     df_payments["shipping_id"]
#     .astype(str)
#     .apply(lambda x: x.split(".")[0] if "." in x else x)
# )
# df_payments = df_payments.drop("shipping", axis=1)
# df_payments.head(1)

In [ ]:
# df_payments["shipping_id"].value_counts()

In [ ]:
# print(df_payments.shape)

In [ ]:
# df_payments = df_payments.drop_duplicates()
# df_payments.sample()

In [ ]:
# # Encontrando os índices das linhas com a data de aprovação mais recente para cada shipping_id
# indices_recentes = df_payments.groupby("shipping_id")["date_approved"].idxmax()
# indices_recentes

In [ ]:
# Verificando se existe mais de um envio
# df_payments["shipping_id"].value_counts()[df_payments["shipping_id"].value_counts() > 1]

In [ ]:
# # Criando um novo DataFrame com base nos índices de envio encontrados
# df_payments = df_payments.loc[indices_recentes]
# df_payments = df_payments.loc[df_payments["shipping_id"] != "nan"]

# df_payments["shipping_id"].value_counts()[df_payments["shipping_id"].value_counts() > 1]

In [ ]:
# print(df_payments.shape)
# df_payments

In [ ]:
# df_payments["payment_status"].value_counts()

In [ ]:
# df_payments["shipping_id"].value_counts()

### Coletando dados de orders

In [ ]:
# df_orders = json_normalize(
#     json_list,
#     record_path=["order_items"],
#     meta=["date_closed", "pack_id", "status", "shipping"],
# )

# ## pd.set_option('display.max_columns', None)
# df_orders.columns

In [ ]:
# df_orders["item.variation_id"].value_counts()

In [ ]:
# cols = [
#     "quantity",
#     "item.id",
#     "item.title",
#     "item.category_id",
#     "item.variation_id",
#     "item.seller_sku",
#     "date_closed",
#     "pack_id",
#     "status",
#     "shipping",
# ]

# df_orders = df_orders[cols]
# df_orders["item.variation_id"] = (
#     df_orders["item.variation_id"]
#     .astype(str)
#     .apply(lambda x: x.split(".")[0] if "." in x else x)
# )

# df_orders.sample(5)

In [ ]:
# x = df_orders[df_orders['item.id'] == 'MLB3387443071']
# x

In [ ]:
# # Extraindo shipping_id
# df_orders["shipping_id"] = df_orders["shipping"].apply(lambda x: x["id"])
# df_orders["shipping_id"] = (
#     df_orders["shipping_id"]
#     .astype(str)
#     .apply(lambda x: x.split(".")[0] if "." in x else x)
# )
# df_orders = df_orders.drop("shipping", axis=1)

In [ ]:
# df_orders.sample(10)

In [ ]:
# valores_unicos = df_orders["shipping_id"].value_counts()
# valores_unicos

In [ ]:
# df_orders.shape

### Unindo DFs de pagamentos e vendas

In [ ]:
# df_resultado = pd.merge(df_orders, df_payments, on="shipping_id", how="left")
# df_resultado.sample(5)

In [ ]:
# df_resultado.shape

In [ ]:
# cols = {
#     "item.id": "ml_code",
#     "item.category_id": "category_id",
#     "item.variation_id": "variation_id",
#     "item.seller_sku": "seller_sku",
#     "item.title": "title",
#     "status": "order_status",
# }
# df_resultado.rename(columns=cols, inplace=True)

# # cols = ['ml_code',  'payment_status', 'shipping_id', 'title', 'variation_id',
# #        'seller_sku', 'quantity', 'fulfilled', 'date_approved', 'date_closed']
# # #  'variation_name', 'variation_value_id',       'variation_value_name',
# # df_resultado = df_resultado[cols]
# df_resultado.sample()

In [ ]:
# df.sample()
# # df.columns

In [ ]:
# print(df_resultado.shape)
# df_resultado = df_resultado.dropna(subset=["date_approved"])
# print(df_resultado.shape)

## Lendo a tabela

In [ ]:
# # Buscando histórico de vendas na tabela ml_orders_hist para o período definido
# try:
#     conn = psycopg2.connect(**db_config)

#     # Construa a consulta SQL com a condição de data
#     sql_query = f"SELECT * FROM ml_orders_hist"
#     print(sql_query)
#     # Execute a consulta e leia os dados em um DataFrame
#     df_orders = pd.read_sql(sql_query, conn)

# except psycopg2.Error as e:
#     print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
#     # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

# except Exception as e:
#     print(f"Erro ao consultar ml_orders_hist: {e}")
#     # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

# finally:
#     if conn is not None:
#         conn.close()

In [ ]:
# df_orders.shape

In [ ]:
# x = "42866524966"
# y = df_resultado[df_resultado["shipping_id"] == x]
# y

In [ ]:
# print(df_resultado.shape)
# df_orders.shape

In [ ]:
# print(df_orders.shape)
# df_orders.sort_values(by="date_closed", ascending=False).head(3)

In [ ]:
# x = "42866524966"
# y = df_orders[df_orders["shipping_id"] == x]
# y

### Populando a tabela

In [ ]:
# cols = [
#     "ml_code",
#     "category_id",
#     "variation_id",
#     "seller_sku",
#     "pack_id",
#     "quantity",
#     "title",
#     "order_status",
#     "payment_status",
#     "shipping_id",
#     "date_approved",
#     "date_closed",
#     "fulfilled",
# ]
# df_resultado = df_resultado[cols]
# df_resultado.columns

In [ ]:
# df_resultado.sort_values(by="date_closed", ascending=False).head(3)

In [ ]:
# df_orders.sort_values(by="date_closed", ascending=True).head(3)

In [ ]:
# conn = psycopg2.connect(**db_config)

# cursor = conn.cursor()

# for index, row in df_orders.iterrows():
# # for index, row in df_resultado.iterrows():
#     insert_query = sql.SQL(
#         "INSERT INTO ml_orders_hist (ml_code,category_id,variation_id,seller_sku,pack_id,quantity,title,order_status,payment_status,shipping_id,date_approved,date_closed,fulfilled) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s)"
#     )
#     cursor.execute(
#         insert_query,
#         (
#                 row["ml_code"],
#                 row["category_id"],
#                 row["variation_id"],
#                 row["seller_sku"],
#                 row["pack_id"],
#                 row["quantity"],
#                 row["title"],
#                 row["order_status"],
#                 row["payment_status"],
#                 row["shipping_id"],
#                 row["date_approved"],
#                 row["date_closed"],
#                 row["fulfilled"],
#         ),
#     )

# conn.commit()

# # Feche o cursor e a conexão
# cursor.close()
# conn.close()
# print("Dados inseridos com sucesso!")

In [12]:
# df['variation_attributes_id'].value_counts()
# df['variation_name'].value_counts()
# df['variation_value_id'].value_counts()
# df['variation_value_name'].value_counts()

variation_attributes_id
COLOR                              22
DRUMSTICK_MALLET_AND_BRUSH_SIZE     2
Name: count, dtype: int64

In [ ]:
# df.columns

In [ ]:
# conn = psycopg2.connect(**db_config)

# cursor = conn.cursor()

# for index, row in df.iterrows():
# # for index, row in df_resultado.iterrows():
#     # insert_query = sql.SQL(
#     #     "INSERT INTO ml_orders_hist (ml_code,category_id,variation_id,seller_sku,pack_id,quantity,title,order_status,payment_status,shipping_id,date_approved,date_closed,fulfilled) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s)"
#     # )
#     insert_query = sql.SQL(
#         "INSERT INTO ml_orders (ml_code,category_id,variation_id,seller_sku,pack_id,quantity,title,order_status,payment_status,variation_name,variation_attributes_id,variation_value_id,variation_value_name,shipping_id,date_approved,date_closed,fulfilled) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)"
#     )
#     cursor.execute(
#         insert_query,
#         (
#                 row["ml_code"],
#                 row["category_id"],
#                 row["variation_id"],
#                 row["seller_sku"],
#                 row["pack_id"],
#                 row["quantity"],
#                 row["title"],
#                 row["order_status"],
#                 row["payment_status"],
#                 row["variation_name"],
#                 row["variation_attributes_id"],
#                 row["variation_value_id"],
#                 row["variation_value_name"],
#                 row["shipping_id"],
#                 row["date_approved"],
#                 row["date_closed"],
#                 row["fulfilled"],
#         ),
#     )

# conn.commit()

# # Feche o cursor e a conexão
# cursor.close()
# conn.close()
# print("Dados inseridos com sucesso!")